In [60]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from sklearn.decomposition import LatentDirichletAllocation as LDA
from nltk.corpus import wordnet as wn
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [61]:
# CSV 파일에서 데이터 불러오기
apps_data = pd.read_csv('cleaned_apps_data.csv')

In [62]:
# 2. 앱 이름을 불용어 리스트로 처리
app_names = apps_data['Name'].apply(lambda x: x.lower()).tolist()  # 앱 이름을 소문자로 변환하여 리스트로 수집


In [63]:
# 3. 추가적인 불용어 정의
additional_stopwords = ['pdf', 'file', 'app', 'lite', 'application', 'https', 'url', 'www']

In [64]:
# 4. 기본 불용어와 추가 불용어 결합
stop_words = list(text.ENGLISH_STOP_WORDS.union(app_names).union(additional_stopwords))


In [65]:
# 2. Sentence-BERT 모델 로드 및 문장 임베딩 생성
model = SentenceTransformer('all-MiniLM-L6-v2')  # Sentence-BERT 모델
embeddings = model.encode(apps_data['Cleaned_Description'].values)  # 앱 설명에 대한 임베딩 생성


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [66]:
# 2. 사용자 입력 클러스터 개수 설정
n_clusters = 5

# 4. K-Means 클러스터링 수행 (사용자가 입력한 클러스터 개수 사용)
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(embeddings)


c:\Users\jeleez\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [67]:
# 5. 클러스터 결과를 데이터프레임에 추가
apps_data['Cluster'] = clusters


In [68]:
# WordNet을 사용하여 키워드의 상위 개념을 찾는 함수
def get_hypernym(word):
    synsets = wn.synsets(word)
    if synsets:
        hypernyms = synsets[0].hypernyms()
        if hypernyms:
            return hypernyms[0].lemmas()[0].name()  # 상위 개념 반환
    return word  # 상위 개념이 없으면 원래 단어 반환

In [69]:
# TF-IDF 기반 주요 키워드 추출 함수
def get_top_keywords(tfidf_matrix, clusters, vectorizer, top_n=5):
    cluster_centers = np.zeros((np.unique(clusters).size, tfidf_matrix.shape[1]))
    
    for cluster in np.unique(clusters):
        cluster_centers[cluster] = tfidf_matrix[clusters == cluster].mean(axis=0)
    
    terms = vectorizer.get_feature_names_out()
    top_keywords = []
    
    for cluster in range(cluster_centers.shape[0]):
        center = cluster_centers[cluster]
        top_indices = center.argsort()[::-1][:top_n]
        keywords = [terms[i] for i in top_indices]
        top_keywords.append(keywords)
    
    return top_keywords


In [70]:
# 6. N-gram 기반 TF-IDF 설정 (bigram 사용)
vectorizer_ngram = TfidfVectorizer(stop_words='english', max_features=1000, ngram_range=(2, 2))
tfidf_matrix_ngram = vectorizer_ngram.fit_transform(apps_data['Description'])


In [71]:
# 7. LDA 기반 주제 모델링
lda = LDA(n_components=n_clusters, random_state=42)
lda.fit(tfidf_matrix_ngram)

def get_lda_topics(lda_model, vectorizer, top_n=5):
    terms = vectorizer.get_feature_names_out()
    topics = []
    for idx, topic in enumerate(lda_model.components_):
        top_indices = topic.argsort()[::-1][:top_n]
        topic_keywords = [terms[i] for i in top_indices]
        topics.append(topic_keywords)
    return topics


In [72]:
# 8. 클러스터별 대표 단어 선택 및 WordNet으로 일반화
def select_representative_word(keywords):
    keyword_embeddings = model.encode(keywords)
    similarity_matrix = cosine_similarity(keyword_embeddings)

    # 각 단어의 유사도 합 계산
    similarity_sums = similarity_matrix.sum(axis=1)
    
    # 유사도 합이 가장 큰 단어를 대표 단어로 선택하고, 상위 개념으로 변환
    representative_idx = np.argmax(similarity_sums)
    representative_word = keywords[representative_idx]
    return get_hypernym(representative_word)

In [73]:
# 9. 세 가지 방법으로 클러스터 대표 단어 생성
# WordNet 기반 일반화
top_keywords_per_cluster = get_top_keywords(tfidf_matrix_ngram, clusters, vectorizer_ngram, top_n=5)
wordnet_based_results = [select_representative_word(keywords) for keywords in top_keywords_per_cluster]

# LDA 기반 토픽 모델링
lda_topics = get_lda_topics(lda, vectorizer_ngram, top_n=5)
lda_based_results = [select_representative_word(topic) for topic in lda_topics]

# N-gram 기반 대표 단어
ngram_based_results = [select_representative_word(keywords) for keywords in top_keywords_per_cluster]


In [74]:
# N-gram 기반 대표 단어
ngram_based_results = [select_representative_word(keywords) for keywords in top_keywords_per_cluster]


In [75]:
# 10. 각 방법에 대한 Silhouette Score 계산
wordnet_silhouette = silhouette_score(embeddings, clusters)
lda_silhouette = silhouette_score(lda.transform(tfidf_matrix_ngram), clusters)
ngram_silhouette = silhouette_score(tfidf_matrix_ngram.toarray(), clusters)


In [76]:
# 11. 결과 출력
print("WordNet 기반 클러스터 대표 단어:", wordnet_based_results)
print(f"WordNet Silhouette Score: {wordnet_silhouette:.3f}\n")

print("LDA 기반 클러스터 대표 단어:", lda_based_results)
print(f"LDA Silhouette Score: {lda_silhouette:.3f}\n")

print("N-gram 기반 클러스터 대표 단어:", ngram_based_results)
print(f"N-gram Silhouette Score: {ngram_silhouette:.3f}\n")

WordNet 기반 클러스터 대표 단어: ['https play', 'mobile phone', 'real time', 'pdf reader', 'https www']
WordNet Silhouette Score: 0.117

LDA 기반 클러스터 대표 단어: ['play google', 'free version', 'privacy policy', 'access rights', 'pdf reader']
LDA Silhouette Score: -0.077

N-gram 기반 클러스터 대표 단어: ['https play', 'mobile phone', 'real time', 'pdf reader', 'https www']
N-gram Silhouette Score: 0.028

